# **LIBRARY IMPORT**

In [1]:
!pip3 install pycryptodome
!pip install cryptography
!pip install pymysql

     |████████████████████████████████| 1.9 MB 14.2 MB/s 
     |████████████████████████████████| 3.5 MB 14.3 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 


In [2]:
import sqlite3
import Crypto 
import binascii
import pandas as pd
import hashlib
import pymysql
from cryptography.fernet import Fernet
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from google.colab import files

# **DATASET**

Penjelasan :
*   Dataset yang diambil adalah sebuah dataset tentang sensor untuk mengecek kualitas air pantai di chicago 
*   Kolom yang diambil dalam dataset ini adalah "Measurement Timestamp", "Water Temperature", "Turbidity", "Transducer Depth", "Wave Height", "Wave Period", "Battery Life"
*   Jumlah record dataset ada 100



In [ ]:
## Read csv dataset
df = pd.read_csv('https://firebasestorage.googleapis.com/v0/b/dataset-476f2.appspot.com/o/beach-water-quality-automated-sensors-1.csv?alt=media&token=ef3a2566-9f96-4236-8008-7e6a1d2dc470')

## Mengambil kolom yg digunakan
data1 = df[["Beach Name", "Measurement Timestamp", "Water Temperature","Turbidity", "Transducer Depth","Wave Height","Wave Period","Battery Life"]]
data2 = data1.rename({'Beach Name' : 'Beach_Name', 'Measurement Timestamp': 'Measurement_Timestamp', 'Water Temperature' : 'Water_Temperature', 'Transducer Depth' : 'Transducer_Depth', 'Wave Height' : 'Wave_Height', 'Wave Period' : 'Wave_Period', 'Battery Life' : 'Battery_Life'}, axis='columns')

## Mengambil 100 data
data = data2.head(100)

## Menambahkan kolom Block id (Untuk Primary key dan Foreign Key)
data = data.drop('Beach_Name', axis = 1)
data.reset_index(inplace=True)
data.rename(columns={'index':'Block_id'}, inplace=True)

Penjelasan:
*   Melakukan proses insert nilai yang kosong dikarenakan pada kolom 'Transducer_Depth' terdapat baris yang tidak ada nilainya
*   Melakukan convert type data dari float ke string. Dikarenakan setiap kolom nilainnya akan di convert menjadi hexa



In [ ]:
## Insert Nilai Kosong
min = data['Transducer_Depth'].min()
data['Transducer_Depth'].fillna(min, inplace = True)

## Convert float to string
data['Water_Temperature'] = data['Water_Temperature'].apply(str)
data['Turbidity'] = data['Turbidity'].apply(str)
data['Transducer_Depth'] = data['Transducer_Depth'].apply(str)
data['Wave_Height'] = data['Wave_Height'].apply(str)
data['Wave_Period'] = data['Wave_Period'].apply(str)
data['Battery_Life'] = data['Battery_Life'].apply(str) 

# **DATA STREM HEAD HASH**

Penjelasan:
*   Melakukan proses hashing untuk setiap nama kolom dataset kedalam SHA1
*   Melakukan proses dari hasil hash di convert menjadi hexa



In [ ]:
## Proses hash to SHA1
column1 = hashlib.sha1(b'Water_Temperature').hexdigest()
column2 = hashlib.sha1(b'Turbidity').hexdigest()
column3 = hashlib.sha1(b'Transducer_Depth').hexdigest()
column4 = hashlib.sha1(b'Wave_Height').hexdigest()
column5 = hashlib.sha1(b'Wave_Period').hexdigest()
column6 = hashlib.sha1(b'Battery_Life').hexdigest()

## Proses hash to hexa
kolom1 = hex(int(column1, 16) + 0x200)[2:]
kolom2 = hex(int(column2, 16) + 0x200)[2:]
kolom3 = hex(int(column3, 16) + 0x200)[2:]
kolom4 = hex(int(column4, 16) + 0x200)[2:]
kolom5 = hex(int(column5, 16) + 0x200)[2:]
kolom6 = hex(int(column6, 16) + 0x200)[2:]

# **DATA STREAM HASH**

Penjelasan:
*   Melakukan proses hashing nilai pada setiap kolom kedalam SHA1
*   Melakukan proses encryp dari hasil hash
*   Melakukan proses dari hasil encryp di convert menjadi hexa





In [ ]:
# ## Proses hash to SHA1
data['Water_Temperature'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Water_Temperature']]
data['Turbidity'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Turbidity']]
data['Transducer_Depth'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Transducer_Depth']]
data['Wave_Height'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Wave_Height']]
data['Wave_Period'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Wave_Period']]
data['Battery_Life'] = [hashlib.sha1(val.encode('UTF-8')).hexdigest() for val in data['Battery_Life']]

## Proses encryption
key = Fernet.generate_key()
f = Fernet(key)

data['Water_Temperature'] = [val.encode() for val in data['Water_Temperature']]
data['Turbidity'] = [val.encode() for val in data['Turbidity']]
data['Transducer_Depth'] = [val.encode() for val in data['Transducer_Depth']]
data['Wave_Height'] = [val.encode() for val in data['Wave_Height']]
data['Wave_Period'] = [val.encode() for val in data['Wave_Period']]
data['Battery_Life'] = [val.encode() for val in data['Battery_Life']]

data['Water_Temperature'] = [f.encrypt(val) for val in data['Wave_Height']]
data['Turbidity'] = [f.encrypt(val) for val in data['Turbidity']]
data['Transducer_Depth'] = [f.encrypt(val) for val in data['Transducer_Depth']]
data['Wave_Height'] = [f.encrypt(val) for val in data['Wave_Height']]
data['Wave_Period'] = [f.encrypt(val) for val in data['Wave_Period']]
data['Battery_Life'] = [f.encrypt(val) for val in data['Battery_Life']]

## Proses convert to hexa
data['Water_Temperature'] = [binascii.hexlify(val) for val in data['Water_Temperature']]
data['Turbidity'] = [binascii.hexlify(val) for val in data['Turbidity']]
data['Transducer_Depth'] = [binascii.hexlify(val) for val in data['Transducer_Depth']]
data['Wave_Height'] = [binascii.hexlify(val) for val in data['Wave_Height']]
data['Wave_Period'] = [binascii.hexlify(val) for val in data['Wave_Period']]
data['Battery_Life'] = [binascii.hexlify(val) for val in data['Battery_Life']]

data['Water_Temperature'] = [val.decode("ascii") for val in data['Water_Temperature']]
data['Turbidity'] = [val.decode("ascii") for val in data['Turbidity']]
data['Transducer_Depth'] = [val.decode("ascii") for val in data['Transducer_Depth']]
data['Wave_Height'] = [val.decode("ascii") for val in data['Wave_Height']]
data['Wave_Period'] = [val.decode("ascii") for val in data['Wave_Period']]
data['Battery_Life'] = [val.decode("ascii") for val in data['Battery_Life']]

## **PUBLIC KEY & PRIVATE KEY**

In [3]:
## Generate key
keyPair = RSA.generate(1024)

## Public key
pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n).lstrip('0x')}, e={hex(pubKey.e).lstrip('0x')})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))

## Private key
print(f"Private key: (n={hex(pubKey.n).lstrip('0x')}, d={hex(keyPair.d).lstrip('0x')})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))

Public key:  (n=c3ee16059defe6c716a6581b7c580b30b51e9cc9b793e09b004119107ec45aaa6733f0c15d1745f267c381e17d5de5117c406f9b9aead1fa9d89caf72edab51298aeecd6e49c779e3255a677f1b6c9bbbceb9d60c0a63cec3079b8c334160e4fabf4691e8bf309bb6a9a3d6f09c658699d4ee7b908626524e72baa118f6c7439, e=10001)
-----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDD7hYFne/mxxamWBt8WAswtR6c
ybeT4JsAQRkQfsRaqmcz8MFdF0XyZ8OB4X1d5RF8QG+bmurR+p2Jyvcu2rUSmK7s
1uScd54yVaZ38bbJu7zrnWDApjzsMHm4wzQWDk+r9Gkei/MJu2qaPW8JxlhpnU7n
uQhiZSTnK6oRj2x0OQIDAQAB
-----END PUBLIC KEY-----
Private key: (n=c3ee16059defe6c716a6581b7c580b30b51e9cc9b793e09b004119107ec45aaa6733f0c15d1745f267c381e17d5de5117c406f9b9aead1fa9d89caf72edab51298aeecd6e49c779e3255a677f1b6c9bbbceb9d60c0a63cec3079b8c334160e4fabf4691e8bf309bb6a9a3d6f09c658699d4ee7b908626524e72baa118f6c7439, d=751ef57b16163a55f86c276f06bd7ab2fb11f9977b6da26e0590e472bfd4907e1c8327ffeeade2da2cbce9fee8fa1fdc7bf1e8c593ad9b27bcd489c788f2a41a14c1a473af8d3f344111d5ab410fd306d1747741c

# **CONCAT DATASET**

Melakukan penggabungan nilai data stream head hash, data stream hash dan public key

In [ ]:
n = hex(pubKey.n).lstrip('0x')
n = str(n)
print(n)

data['Water_Temperature'] = kolom1 + data['Water_Temperature'] + n 
data['Turbidity'] = kolom2 + data['Turbidity'] + n
data['Transducer_Depth'] = kolom3 + data['Transducer_Depth'] + n
data['Wave_Height'] = kolom4 + data['Wave_Height'] + n
data['Wave_Period'] = kolom5 + data['Wave_Period'] + n
data['Battery_Life'] = kolom6 + data['Battery_Life'] + n

## 256 : Public key
## 40 : Data stream head hash
## 280 : Data stream hash
## Total 288 byte

Menggabungkan semua kolom dataset menjadi 1 kolom

In [ ]:
## Membuat variable untuk menampung setiap kolom dataset
dataset1 = data[["Block_id", "Water_Temperature"]]
dataset1.rename(columns={'Water_Temperature':'Data'}, inplace=True)

dataset2 = data[["Block_id", "Turbidity"]]
dataset2.rename(columns={'Turbidity':'Data'}, inplace=True)

dataset3 = data[["Block_id", "Transducer_Depth"]]
dataset3.rename(columns={'Transducer_Depth':'Data'}, inplace=True)

dataset4 = data[["Block_id", "Wave_Height"]]
dataset4.rename(columns={'Wave_Height':'Data'}, inplace=True)

dataset5 = data[["Block_id", "Wave_Period"]]
dataset5.rename(columns={'Wave_Period':'Data'}, inplace=True)

dataset6 = data[["Block_id", "Battery_Life"]]
dataset6.rename(columns={'Battery_Life':'Data'}, inplace=True)

## Menggabungkan dataset menjadi 1 kolom
dataset = pd.concat([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6], ignore_index=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# **MENGELOLA DATASET**

In [ ]:
## Ascending dataset sesuai Block_id
dataa = dataset.sort_values(by=['Block_id'])

## Convert to csv
dataa.to_csv('dataset.csv', index=False)
data1 = pd.read_csv('dataset.csv')

Melakukan penambahan kolom indeks data

In [ ]:
## Membuat index blockchain
index_data = df['Measurement ID']
index_data = index_data.head(100)

## Menggabungkan dengan index dataset
dataset = pd.concat([index_data, data1], axis = 1)

## Menambahkan kolom Indeks data
dataset = dataset.drop('Measurement ID', axis = 1)
dataset.reset_index(inplace=True)
dataset.rename(columns={'index':'Indeks_data'}, inplace=True)

# **CONVERT TO CSV**

In [ ]:
## Dataset Table 1
data = data[["Block_id", "Measurement_Timestamp"]]
data.to_csv('table1.csv', index=False)
files.download('table1.csv')

## Dataset table 2
dataset.to_csv('table2.csv', index=False)
files.download('table2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **DATABASE**

In [ ]:
## Connect database localhost/phhpmyadmin/bloomfilter
conn = pymysql.connect(host="localhost", user="root", passwd="", db="bloomfilter" )
cursor = conn.cursor()

## Membuat table 1
cursor.execute("""CREATE TABLE Dataset1(
    Block_id INT PRIMARY KEY,
    Measurement_Timestamp TEXT
    )
    """)

# Membuat table 2
cursor.execute("""CREATE TABLE Dataset2(
    Index_data INT PRIMARY KEY,
    Block_id INT,
    Data TEXT
    )
    """)

cursor.execute( """ALTER TABLE Dataset2(add foreign key (Block_id) references Dataset1(Block_id)""")

# Mengisi data table 1
data.to_sql(con = conn, name = 'Dataset1', if_exists='replace', index = False)

## Mengisi data table 2
dataset.to_sql('Dataset2', conn, if_exists='replace', index = False)

## Close database
conn.commit()
conn.close()